In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
import os
import cvxpy as cp
from cvxopt import matrix, solvers
import sklearn.covariance as sk_cov
import pyRMT as rmt
from pdb import set_trace as bp

In [3]:
home_dir = os.path.expandvars("$HOME")
app_src_dir = '/dev/repos/FinancialAnalyticsSystem/src'
sys.path.insert(0, home_dir + app_src_dir)

In [4]:
from DataHandlingSystem.DataLoadingSystem import *
from FeatureExtractionSystem.DeltaFeatureExtractor import * 

In [6]:
data = pd.read_pickle('data.pckl')
data

,F,L,MSFT,S,NVDA,ANSS,VZ,WMT,MRK,EQV1V_HE,...,UTHR,UFPI,URBN,VRSN,VRTX,VSAT,WERN,WRLD,XLNX,ZBRA
date,,,,,,,,,,,,,,,,,,,,,
2000-11-02 00:00:00+02:00,15.048508,11.346896,22.8218,17.588865,10.2844,2.9075,21.8678,34.0712,43.3611,0.963153,...,33.000000,3.909851,1.109375,115.378403,97.250000,21.875000,4.686553,5.031200,50.298553,19.027779
2000-11-03 00:00:00+02:00,14.615457,10.939939,22.1531,18.663744,10.9564,2.9075,21.5838,33.3256,42.7529,1.016662,...,35.500000,3.963903,1.031250,120.897881,93.875000,22.000000,4.666527,5.031200,51.515450,20.194445
2000-11-06 00:00:00+02:00,15.265035,11.083570,22.5589,18.321737,11.8021,2.9375,21.0385,34.4650,43.7843,1.177187,...,35.000000,4.035973,1.007812,121.654137,91.687500,19.000000,4.666527,5.062500,52.552063,20.055555
2000-11-07 00:00:00+02:00,14.435014,11.330938,22.8835,19.298891,11.0040,2.8125,20.8038,34.4228,42.2665,1.230696,...,35.000000,3.914347,1.003900,118.668861,96.859299,20.125000,4.666527,5.062500,48.090099,20.388889
2000-11-08 00:00:00+02:00,14.326763,11.474570,22.5394,19.885189,10.6780,2.8750,20.8492,34.2892,44.1784,1.123679,...,34.875000,3.945885,0.992188,107.948334,93.250000,18.562500,4.706581,5.125000,42.050678,20.027779
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-19 00:00:00+02:00,7.792062,44.709541,102.8713,5.980000,138.2075,146.2200,54.1336,89.5870,72.7918,7.601695,...,104.169998,26.221514,33.060001,149.710007,158.660004,61.930000,26.675495,96.930000,84.371185,159.039993
2018-12-20 00:00:00+02:00,7.735870,44.480415,100.7085,5.980000,134.8050,143.8000,53.3680,86.3518,72.5155,7.544683,...,103.040001,25.674818,32.770000,147.020004,161.300003,61.419998,26.543133,97.120003,82.648514,150.990005
2018-12-21 00:00:00+02:00,7.539195,43.703381,97.4544,5.790000,129.2870,140.1900,53.2226,86.2034,71.9333,7.582691,...,102.510002,25.833857,31.879999,142.440002,156.500000,58.720001,25.943090,94.650002,80.698135,146.250000


In [12]:
index = 0
k = 10
while index < len(data.columns):
    print(data.columns[index: index + k])
    index += k

Index(['F', 'L', 'MSFT', 'S', 'NVDA', 'ANSS', 'VZ', 'WMT', 'MRK', 'EQV1V_HE'], dtype='object')
Index(['SAMPO_HE', 'FORTUM_HE', 'UPM_HE', 'MC_PA', 'AAPL', 'A', 'C', 'D',
       'AMWD', 'ABCB'],
      dtype='object')
Index(['AMGN', 'INTC', 'CMCSA', 'PEP', 'ASML', 'KO', 'DIS', 'IBM', 'JNJ',
       'FLWS'],
      dtype='object')
Index(['ADBE', 'AEIS', 'MMM', 'AXP', 'BA', 'CAT', 'CSCO', 'AKAM', 'ALXN',
       'ALKS'],
      dtype='object')
Index(['ARLP', 'AMZN', 'STERV_HE', 'BITTI_HE', 'DAI_DE', 'MCD', 'JPM', 'NKE',
       'PFE', 'PG'],
      dtype='object')
Index(['UTX', 'AMKR', 'AMAT', 'ARQL', 'AZPN', 'BANF', 'BBBY', 'BMRN', 'EPAY',
       'BRKS'],
      dtype='object')
Index(['CHRW', 'CALM', 'CECO', 'CWST', 'CASY', 'CELG', 'CETV', 'CENT', 'CERN',
       'CHKP'],
      dtype='object')
Index(['PLCE', 'CTAS', 'CRUS', 'CTXS', 'COKE', 'CGNX', 'CTSH', 'COLB', 'COLM',
       'CNMD'],
      dtype='object')
Index(['CPRT', 'CSGP', 'CBRL', 'CACC', 'CREE', 'CSGS', 'CY', 'XRAY', 'DISH',
       'DLTR'

In [13]:
data.index.size

4409